In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/kaggle/input/drug-classification/drug200.csv')
df.head()

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False)

In [ ]:
sns.countplot(x='Sex', hue='Drug', data=df, palette='rainbow')

In [ ]:
sns.countplot(x='Cholesterol', hue='Drug',data=df)

In [ ]:
sns.countplot(x='BP', hue='Drug',data=df)

In [ ]:
plt.figure(figsize=(12,7))
sns.boxplot(x='Drug', y='Age', data=df, showmeans=True)

In [ ]:
sns.boxplot(x='Drug', y='Na_to_K', data=df, hue='BP', showmeans=True)

In [ ]:
sns.boxplot(x='Drug', y='Na_to_K', data=df, hue='Cholesterol', showmeans=True)

In [ ]:
sns.boxplot(x='Drug', y='Na_to_K', data=df, hue='Sex', showmeans=True)

In [ ]:
sns.countplot(x='Cholesterol', hue='Sex',data=df)
#sns.countplot(x='BP', hue='Sex',data=df)

In [ ]:
sns.displot(data= df, x='Na_to_K', kind='kde')
print(df.var())

In [ ]:
sns.displot(data= df, x='Age', hue='Drug', kind='kde')

In [ ]:
#df['Na_to_K'] = np.log(df['Na_to_K'])
sns.displot(data= df, x='Na_to_K', kind='kde')

In [ ]:
# from sklearn.preprocessing import StandardScaler
# std = StandardScaler()
# subset = df[['Na_to_K','Age']]
# scaled_ss = std.fit_transform(subset)
# df.drop(columns=['Na_to_K','Age'], inplace=True)
# df = pd.concat([df, pd.DataFrame(scaled_ss, columns=['Na_to_K','Age'])], axis=1)
# print(df.head())
# print(df.var())


In [ ]:
bp = pd.get_dummies(df['BP'], drop_first=True, prefix='BP_')
cholesterol = pd.get_dummies(df['Cholesterol'], drop_first=True,prefix='CHOL_')
df_new = df.drop(columns=['Sex','BP','Cholesterol'])
df_new = pd.concat([df_new, bp, cholesterol], axis=1)
df_new.head()

In [ ]:
from  sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = pd.DataFrame(scaler.fit_transform(df_new.drop(columns=['Drug'])), columns=df_new.drop(columns=['Drug']).columns)
print(x.var())
sns.displot(data= df, x='Na_to_K', kind='kde')

In [ ]:
from sklearn.model_selection import train_test_split
y = df_new['Drug']
x = df_new.drop(columns=['Drug'])
x_train , x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
x_train.head()

In [ ]:
df_new.corr()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(penalty='l1', solver='liblinear', multi_class='ovr')
log_model.fit(x_train, y_train)
y_pred = log_model.predict(x_test)
y_pred_prob = log_model.predict_proba(x_test)

acc = accuracy_score(y_test, y_pred)
acc

In [ ]:
print(y_pred_prob[:,1])
# print(roc_auc_score(y_test, y_pred_prob[:,1],multi_class='ovr',  average="weighted"))

In [ ]:
from sklearn.metrics import classification_report
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=log_model.predict(x_test))
print(sk_report)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_model = DecisionTreeClassifier(criterion='gini', max_depth=4)
tree_model.fit(x_train, y_train)
y_pred = tree_model.predict(x_test)
acc = accuracy_score(y_test, y_pred)
acc

In [ ]:
from sklearn.metrics import classification_report
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=tree_model.predict(x_test))
print(sk_report)